In [42]:
%pip install altair numpy pandas matplotlib seaborn scikit-learn

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [43]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)

# **Phase 1: Reading and Cleaning the Data**

In [44]:
sales_product = pd.read_csv("/Users/aseemarora08/Desktop/PersonalProject/PersonalProjects/sales_data.csv")
sales_product

,Unnamed: 0,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour
0,0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0
1,1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7
2,2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,New York City,18
3,3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15
4,4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12
...,...,...,...,...,...,...,...,...,...,...,...
185945,13617,222905,AAA Batteries (4-pack),1,2.99,2019-06-07 19:02:00,"795 Pine St, Boston, MA 02215",6,2.99,Boston,19
185946,13618,222906,27in FHD Monitor,1,149.99,2019-06-01 19:29:00,"495 North St, New York City, NY 10001",6,149.99,New York City,19
185947,13619,222907,USB-C Charging Cable,1,11.95,2019-06-22 18:57:00,"319 Ridge St, San Francisco, CA 94016",6,11.95,San Francisco,18
185948,13620,222908,USB-C Charging Cable,1,11.95,2019-06-26 18:35:00,"916 Main St, San Francisco, CA 94016",6,11.95,San Francisco,18


In [45]:
sales_product.dtypes

Unnamed: 0            int64
Order ID              int64
Product              object
Quantity Ordered      int64
Price Each          float64
Order Date           object
Purchase Address     object
Month                 int64
Sales               float64
City                 object
Hour                  int64
dtype: object

In [46]:
# Convert 'Order Date' to datetime format
sales_product['Order Date'] = pd.to_datetime(sales_product['Order Date'], format='%Y-%m-%d %H:%M:%S')

# Print the updated data types of each column
data_types_updated = sales_product.dtypes
print(data_types_updated)

Unnamed: 0                   int64
Order ID                     int64
Product                     object
Quantity Ordered             int64
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
Month                        int64
Sales                      float64
City                        object
Hour                         int64
dtype: object


In [47]:
data_types_updated

Unnamed: 0                   int64
Order ID                     int64
Product                     object
Quantity Ordered             int64
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
Month                        int64
Sales                      float64
City                        object
Hour                         int64
dtype: object

In [48]:
sales_product

,Unnamed: 0,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour
0,0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0
1,1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7
2,2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,New York City,18
3,3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15
4,4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12
...,...,...,...,...,...,...,...,...,...,...,...
185945,13617,222905,AAA Batteries (4-pack),1,2.99,2019-06-07 19:02:00,"795 Pine St, Boston, MA 02215",6,2.99,Boston,19
185946,13618,222906,27in FHD Monitor,1,149.99,2019-06-01 19:29:00,"495 North St, New York City, NY 10001",6,149.99,New York City,19
185947,13619,222907,USB-C Charging Cable,1,11.95,2019-06-22 18:57:00,"319 Ridge St, San Francisco, CA 94016",6,11.95,San Francisco,18
185948,13620,222908,USB-C Charging Cable,1,11.95,2019-06-26 18:35:00,"916 Main St, San Francisco, CA 94016",6,11.95,San Francisco,18


In [49]:

# Extract 'Order Date' and 'Order Time' components
sales_product['Order Time'] = sales_product['Order Date'].dt.time
sales_product['Order Date'] = sales_product['Order Date'].dt.date


# Print the first few rows of the DataFrame to check the result
sales_product.head()

,Unnamed: 0,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour,Order Time
0,0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,00:01:00
1,1,295666,LG Washing Machine,1,600.00,2019-12-29,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,07:03:00
2,2,295667,USB-C Charging Cable,1,11.95,2019-12-12,"277 Main St, New York City, NY 10001",12,11.95,New York City,18,18:21:00
3,3,295668,27in FHD Monitor,1,149.99,2019-12-22,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15,15:13:00
4,4,295669,USB-C Charging Cable,1,11.95,2019-12-18,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12,12:38:00


In [50]:
# Drop rows containing NaN values
sales_product = sales_product.dropna()
sales_product

,Unnamed: 0,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour,Order Time
0,0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,00:01:00
1,1,295666,LG Washing Machine,1,600.00,2019-12-29,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,07:03:00
2,2,295667,USB-C Charging Cable,1,11.95,2019-12-12,"277 Main St, New York City, NY 10001",12,11.95,New York City,18,18:21:00
3,3,295668,27in FHD Monitor,1,149.99,2019-12-22,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15,15:13:00
4,4,295669,USB-C Charging Cable,1,11.95,2019-12-18,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12,12:38:00
...,...,...,...,...,...,...,...,...,...,...,...,...
185945,13617,222905,AAA Batteries (4-pack),1,2.99,2019-06-07,"795 Pine St, Boston, MA 02215",6,2.99,Boston,19,19:02:00
185946,13618,222906,27in FHD Monitor,1,149.99,2019-06-01,"495 North St, New York City, NY 10001",6,149.99,New York City,19,19:29:00
185947,13619,222907,USB-C Charging Cable,1,11.95,2019-06-22,"319 Ridge St, San Francisco, CA 94016",6,11.95,San Francisco,18,18:57:00
185948,13620,222908,USB-C Charging Cable,1,11.95,2019-06-26,"916 Main St, San Francisco, CA 94016",6,11.95,San Francisco,18,18:35:00


# **Phase 2: Wrangling**

### Computing Products and their sales

In [51]:
# Creating a copy of the DataFrame to avoid modifying the original
total_sales_product = sales_product.copy()

# Calculating total sales for each product and add a new column 'Total Sales'
total_sales_product = total_sales_product.assign(
    Total_Sales=total_sales_product.groupby('Product')['Quantity Ordered'].transform('sum') * total_sales_product['Price Each']
)

# Print the updated DataFrame
total_sales_product

,Unnamed: 0,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour,Order Time,Total_Sales
0,0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0,00:01:00,8037600.00
1,1,295666,LG Washing Machine,1,600.00,2019-12-29,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7,07:03:00,399600.00
2,2,295667,USB-C Charging Cable,1,11.95,2019-12-12,"277 Main St, New York City, NY 10001",12,11.95,New York City,18,18:21:00,286501.25
3,3,295668,27in FHD Monitor,1,149.99,2019-12-22,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15,15:13:00,1132424.50
4,4,295669,USB-C Charging Cable,1,11.95,2019-12-18,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12,12:38:00,286501.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185945,13617,222905,AAA Batteries (4-pack),1,2.99,2019-06-07,"795 Pine St, Boston, MA 02215",6,2.99,Boston,19,19:02:00,92740.83
185946,13618,222906,27in FHD Monitor,1,149.99,2019-06-01,"495 North St, New York City, NY 10001",6,149.99,New York City,19,19:29:00,1132424.50
185947,13619,222907,USB-C Charging Cable,1,11.95,2019-06-22,"319 Ridge St, San Francisco, CA 94016",6,11.95,San Francisco,18,18:57:00,286501.25
185948,13620,222908,USB-C Charging Cable,1,11.95,2019-06-26,"916 Main St, San Francisco, CA 94016",6,11.95,San Francisco,18,18:35:00,286501.25


In [52]:
total_sales_product = total_sales_product[["Product", "Total_Sales"]]
total_sales_product

,Product,Total_Sales
0,Macbook Pro Laptop,8037600.00
1,LG Washing Machine,399600.00
2,USB-C Charging Cable,286501.25
3,27in FHD Monitor,1132424.50
4,USB-C Charging Cable,286501.25
...,...,...
185945,AAA Batteries (4-pack),92740.83
185946,27in FHD Monitor,1132424.50
185947,USB-C Charging Cable,286501.25
185948,USB-C Charging Cable,286501.25


In [53]:
# Removing duplicate rows based on the 'Product' column
total_sales_product_no_duplicates = total_sales_product.drop_duplicates(subset=['Product'])

# Print the DataFrame without duplicate product names
total_sales_product_no_duplicates

,Product,Total_Sales
0,Macbook Pro Laptop,8037600.00
1,LG Washing Machine,399600.00
2,USB-C Charging Cable,286501.25
3,27in FHD Monitor,1132424.50
5,AA Batteries (4-pack),106118.40
8,Bose SoundSport Headphones,1345565.43
9,AAA Batteries (4-pack),92740.83
11,ThinkPad Laptop,4129958.70
15,Lightning Charging Cable,347094.15
16,Google Phone,3319200.00


In [57]:
# Sort the DataFrame by 'Total_Sales' in descending order
total_sales_product_no_duplicates = total_sales_product_no_duplicates.sort_values(by='Total_Sales', ascending=False)
total_sales_product_no_duplicates

,Product,Total_Sales
0,Macbook Pro Laptop,8037600.00
54,iPhone,4794300.00
11,ThinkPad Laptop,4129958.70
16,Google Phone,3319200.00
143,27in 4K Gaming Monitor,2435097.56
61,34in Ultrawide Monitor,2355558.01
25,Apple Airpods Headphones,2349150.00
89,Flatscreen TV,1445700.00
8,Bose SoundSport Headphones,1345565.43
3,27in FHD Monitor,1132424.50


In [71]:
#Plotting Altair BarChart
total_sales_plot = alt.Chart(total_sales_product_no_duplicates).mark_bar().encode(
    y = alt.Y("Total_Sales:Q").title('Total Sales').axis(format='~s'),
    x = alt.X("Product").title("Product").sort("-y")
)
total_sales_plot

alt.Chart(...)

### Studying Macbook Purchasing Paterns and Trends